In [ ]:
import os
import pickle
import numpy as np
import sys

# Note: Due to the large data size and model weights, we have uploaded DeepMReye related files (data, model weights, model history, results) to an external storage, which can be accessed [here](https://drive.google.com/drive/folders/1v8Et8A2n2PrOwLj3JJYkjh6VXxKMqX5C?usp=drive_link).

sys.path.append('/DATA/publish/mocet/analysis/scripts')
from utils.base import get_minecraft_subjects, get_project_directory, get_configs

subject_pool = get_minecraft_subjects()
project_dir = get_project_directory()
configs = get_configs()

valid_data = pickle.load(open('../../data/valid_data_list.pkl', 'rb'))
valid_keys = list(valid_data.keys())


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from deepmreye import analyse, architecture, preprocess, train
from deepmreye.util import data_generator, model_opts, util

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

experiment_folder = "/DATA/publish/mocet/analysis/data/DeepMReye"
functional_data = os.path.join("/DATA", "Minecraft_old", "_DATA", "fMRI")
processed_data = os.path.join(experiment_folder, "processed_data/")
model_weights = os.path.join(experiment_folder, "model_weights/")
results = os.path.join(experiment_folder, "results/")

if not os.path.exists(processed_data):
    os.makedirs(processed_data)
if not os.path.exists(model_weights):
    os.makedirs(model_weights)
if not os.path.exists(results):
    os.makedirs(results)

task = 'task-mcHERDING'
space = 'space-MNI152NLin2009cAsym'
prefix = 'desc-preproc_bold.nii.gz'

eyemask_small, eyemask_big, dme_template, mask, x_edges, y_edges, z_edges, = preprocess.get_masks()
subjects = subject_pool.keys()
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        root = f'{project_dir}/data/eyetracking/{subject}/{session}'
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)
                fp_func = os.path.join(functional_data, subject, session,
                                       f'{subject}_{session}_{task}_{run}_{space}_{prefix}')
                if os.path.exists(os.path.join(functional_data, subject, session,
                                               f'mask_{subject}_{session}_{task}_{run}_{space}_desc-preproc_bold.p')):
                    print('Already preprocessed')
                else:
                    preprocess.run_participant(
                        fp_func,
                        dme_template,
                        eyemask_big,
                        eyemask_small,
                        x_edges,
                        y_edges,
                        z_edges,
                    )

In [ ]:
calibration_points = np.array([[200-800, 166-500], [200-800, 500-500], [200-800, 833-500],
                               [600-800, 166-500], [600-800, 500-500], [600-800, 833-500],
                               [1000-800, 166-500], [1000-800, 500-500], [1000-800, 833-500],
                               [1400-800, 166-500], [1400-800, 500-500], [1400-800, 833-500]]) * (1/78.0487)
calibration_points = np.tile(calibration_points, [2,1])
calibration_order = np.array([4,11,6,2,7,0,10,5,9,8,1,3]*2)

subjects = subject_pool.keys()
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)
                participant_train_data = []
                participant_train_labels = []
                participant_train_ids = []

                participant_test_data = []
                participant_test_labels = []
                participant_test_ids = []

                # Load mask and normalize it
                # 216 TRs
                this_mask = os.path.join(functional_data, subject, session,
                                         f'mask_{subject}_{session}_{task}_{run}_{space}_desc-preproc_bold.p')
                this_mask = pickle.load(open(this_mask, "rb"))
                this_mask = preprocess.normalize_img(this_mask)
                train_mask = np.copy(this_mask[...,1:25])
                test_mask = np.copy(this_mask)

                # Load labels (in visual angle)
                #this_label = preprocess.load_label(gaze_data, label_type="calibration_run")
                train_label = calibration_points[calibration_order, :]
                train_label = train_label[:, np.newaxis, :]
                train_label = np.repeat(train_label, 10, axis=1)
                test_label = np.zeros((510,10,2))

                # Store across runs
                participant_train_data.append(train_mask)
                participant_train_labels.append(train_label)
                participant_train_ids.append(([subject] * train_label.shape[0],[0] * train_label.shape[0]))

                # Store across run
                participant_test_data.append(test_mask)
                participant_test_labels.append(test_label)
                participant_test_ids.append(([subject] * test_label.shape[0],[0] * test_label.shape[0]))

                # Save participant file
                preprocess.save_data(
                    subject + f'_{session}_{run}_train',
                    participant_train_data,
                    participant_train_labels,
                    participant_train_ids,
                    processed_data,
                    center_labels=False,
                )
                preprocess.save_data(
                    subject + f'_{session}_{run}_test',
                    participant_test_data,
                    participant_test_labels,
                    participant_test_ids,
                    processed_data,
                    center_labels=False,
                )


In [ ]:
# From scratch

import os
from tensorflow.keras.callbacks import Callback

class LossHistoryCallback(Callback):
  def __init__(self, filename):
    super().__init__()
    self.filename = filename
    with open(self.filename, "w") as f:
      f.write("Epoch,Training Loss\n")
  def on_epoch_end(self, epoch, logs=None):
    training_loss = logs.get('loss', 'NA')
    with open(self.filename, "a") as f:
      f.write(f"{epoch+1},{training_loss}\n")

# Training
subjects = subject_pool.keys()
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)

                data = np.load(f'{processed_data}/{subject}_{session}_{run}_train.npz', mmap_mode='r')
                X = [data['data_' + str(b)] for b in range(24)]
                y = [data['label_' + str(b)] for b in range(24)]
                X = np.array(X)[..., np.newaxis]
                y = np.array(y)
                opts = model_opts.get_opts()

                opts["epochs"] = 120
                opts["gaussian_noise"] = 1.0
                lr_sched = util.step_decay_schedule(initial_lr=opts['lr'], decay_factor=0.95, num_epochs=opts["epochs"])

                loss_history_callback = LossHistoryCallback(filename=f'{model_weights}/{subject}_{session}_{run}_loss_scratch.txt')
                model, model_inference = architecture.create_standard_model(X.shape[1::], opts)
                model.fit([X, y],
                          epochs=opts["epochs"],
                          callbacks=[lr_sched, loss_history_callback],
                          use_multiprocessing=True,
                          workers=36)
                model_inference.save_weights(f'{model_weights}/{subject}_{session}_{run}_model_scratch.h5')

# Prediction
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)
                test_data = np.load(f'{processed_data}/{subject}_{session}_{run}_test.npz', mmap_mode='r')

                test_X = [test_data['data_' + str(b)] for b in range(510)]
                test_y = [test_data['label_' + str(b)] for b in range(510)]
                test_X = np.array(test_X)[..., np.newaxis]
                test_y = np.array(test_y)

                opts = model_opts.get_opts()
                _, model_inference = architecture.create_standard_model(test_X.shape[1::], opts)
                model_inference.load_weights(f'{model_weights}/{subject}_{session}_{run}_model_scratch.h5')
                (pred_y, euc_pred) = model_inference.predict(test_X)

                pred_y = np.median(pred_y, axis=1) #Original DeepMReye evaluation funcation used np.median
                np.save(f'{results}/{subject}_{session}_{run}_pred_scratch.npy', pred_y)

In [ ]:
# Finetuned

subjects = subject_pool.keys()
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)

                data = np.load(f'{processed_data}/{subject}_{session}_{run}_train.npz', mmap_mode='r')
                X = [data['data_' + str(b)] for b in range(24)]
                y = [data['label_' + str(b)] for b in range(24)]
                X = np.array(X)[..., np.newaxis]
                y = np.array(y)
                opts = model_opts.get_opts()

                opts["epochs"] = 120
                opts["gaussian_noise"] = 1.0
                lr_sched = util.step_decay_schedule(initial_lr=opts['lr'], decay_factor=0.95, num_epochs=opts["epochs"])

                loss_history_callback = LossHistoryCallback(filename=f'{model_weights}/{subject}_{session}_{run}_loss_finetuned.txt')
                model, model_inference = architecture.create_standard_model(X.shape[1::], opts)
                model.load_weights(f'{model_weights}/pretrained_weight_datasets_1to6.h5')
                model.fit([X, y],
                          epochs=opts["epochs"],
                          callbacks=[lr_sched, loss_history_callback],
                          use_multiprocessing=True,
                          workers=36,
                          )
                model_inference.save_weights(f'{model_weights}/{subject}_{session}_{run}_model_finetuned.h5')

# Prediction
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)
                test_data = np.load(f'{processed_data}/{subject}_{session}_{run}_test.npz', mmap_mode='r')

                test_X = [test_data['data_' + str(b)] for b in range(510)]
                test_y = [test_data['label_' + str(b)] for b in range(510)]
                test_X = np.array(test_X)[..., np.newaxis]
                test_y = np.array(test_y)

                opts = model_opts.get_opts()
                _, model_inference = architecture.create_standard_model(test_X.shape[1::], opts)
                model_inference.load_weights(f'{model_weights}/{subject}_{session}_{run}_model_finetuned.h5')
                (pred_y, euc_pred) = model_inference.predict(test_X)

                pred_y = np.median(pred_y, axis=1) #Original DeepMReye evaluation funcation used np.median
                np.save(f'{results}/{subject}_{session}_{run}_pred_finetuned.npy', pred_y)

In [ ]:
# pretrained

# Prediction
for subject in subjects:
    sessions = subject_pool[subject].keys()
    for session in sessions:
        runs = subject_pool[subject][session]
        for r in runs:
            run = f'run-{r}'
            np.random.seed(0)
            key = (subject, session, task, run)
            if key in valid_keys:
                print(subject, session, task, run)
                test_data = np.load(f'{processed_data}/{subject}_{session}_{run}_test.npz', mmap_mode='r')

                test_X = [test_data['data_' + str(b)] for b in range(510)]
                test_y = [test_data['label_' + str(b)] for b in range(510)]
                test_X = np.array(test_X)[..., np.newaxis]
                test_y = np.array(test_y)

                opts = model_opts.get_opts()
                _, model_inference = architecture.create_standard_model(test_X.shape[1::], opts)
                model_inference.load_weights(f'{model_weights}/pretrained_weight_datasets_1to6.h5')
                (pred_y, euc_pred) = model_inference.predict(test_X)
                pred_y = np.median(pred_y, axis=1) #Original DeepMReye evaluation funcation used np.median
                np.save(f'{results}/{subject}_{session}_{run}_pred_pretrained.npy', pred_y)